<img src='files/bayes.png'/>
<pre>                                                     Bayes Teoremi</pre>

In [ ]:
# Bayes Teoremi elimizdeki öncül bilgilere göre olasılık hesabı yapmamızı sağlar.
# B olayı gerçekleştikten sonra A olayının gerçekleşme olasılığının hesaplanması 
# yukarıdaki gibidir. 

In [1]:
# Bayes Teoremi'nden faydalanarak bir Bayes Sınıflandırıcısı oluşturabiliriz. 
# Bayes Sınıflandırıcısı her bir faktör için olasılıkları hesaplar ve en yüksek olasılığı
# veren faktörü seçer. Girdi özniteliklerinin birbirinden bağımsız olduğu varsayılır.
# Gerçek zamanlı tahmin, metin sınıflandırma, spam filtreleme, öneri sistemleri gibi
# pratik uygulamalarda kullanılmaktadır.

In [2]:
# Öncelikle maillerde bulunan kelimeleri ve bu kelimelerin frekanslarını içeren bir sözlük
# oluşturacağız. Ardından bu sözlük üzerinde bazı kelimeleri sileceğiz. Bu kelimeler analize
# bir katkısı olmayan sayısal değerler ve ya 'this', 'is', 'are' gibi kelimeler olacak. Bunu
# sağlayabilmek için uzunluğu 5 harften az olan kelimeleri ve sayısal değerleri atacağız.
# Son olarak sözlüğümüzdeki en sık 3bin kelimeyi seçerek sözlüğümüzü kısaltacağız.
# Modelimiz adı üstünde naif. Kelimelerin gerçekte birbirinden bağımsız olduğunu varsayıyor.

In [2]:
# Pathleri tanımlayalım
training_path = '../Datasets/Mails-Dataset/training-mails'
test_path     = '../Datasets/Mails-Dataset/test-mails'

In [16]:
import os
from collections import Counter

# Sözlüğümüzü oluşturalım
def create_dictionary(dir):
    all_words = []
    emails = [os.path.join(dir, file) for file in os.listdir(dir)]
    
    for mail in emails:
        with open(mail) as m:
            for line in m:
                all_words += line.split()
                
    dictionary = Counter(all_words)
    words = list(dictionary)
    
    for word in words:
        if word.isalpha() == False:
            del dictionary[word]
        if len(word) < 5:
            del dictionary[word]
            
    # Sözlükte bulunmayan kelimeleri temsil etmek için bir anahtar oluşturalım
    dictionary = dictionary.most_common(3000)
    
    # Kelimeleri anahtar, değerleri ise indis-frekans ikilileri yaptık
    # Böylece her bir kelime unique indise sahip oldu
    dictionary = {dictionary[index][0]:(index, dictionary[index][1]) for index in range(len(dictionary))}
    
    return dictionary

dictionary = create_dictionary(training_path)

In [4]:
# Öznitelik matrisini oluşturalım. Öznitelik matrisinin her bir satırı her bir mail
# için öznitelik vektörüne karşılık gelir. Bu vektör sözlük uzunluğu kadar bileşene
# sahiptir. Bu bileşenlerin her biri bir kelimeye karşılık düşer ve mail içerisinde
# ilgili kelimenin sıklığını verir.

In [17]:
import numpy as np

def create_feature_matrix(dir, dictionary):
    
    # Email pathlerini alalım
    emails = [os.path.join(dir, file) for file in os.listdir(dir)]
    
    # Öznitelik matrisi ve etiket vektörünü başlatalım
    feature_matrix = np.zeros((len(emails), len(dictionary)))
    labels = np.zeros(len(emails))
    
    for doc_id, mail in enumerate(emails):
        with open(mail) as m:
            # İçerik 2. satırda bulunuyor
            content = m.readlines()[2]
            
            words = content.split()
            for word in words:
                if word in dictionary:
                    word_index = dictionary[word][0]
                    freq = words.count(word)
                    feature_matrix[doc_id][word_index] = freq
                
        # Etiketi belirleyelim
        suffix = mail.split('/')[-1]
        if suffix.startswith('spmsg'):
            labels[doc_id] = 1
        else:
            labels[doc_id] = 0
    
    return feature_matrix, labels

In [22]:
# Modelimizi Gauss dağılımını kullanarak kuralım
from sklearn.naive_bayes import GaussianNB

training_features_matrix, training_labels = create_feature_matrix(training_path, dictionary)
test_features_matrix, test_labels = create_feature_matrix(test_path, dictionary)

model = GaussianNB()
model.fit(training_features_matrix, training_labels)

# Tahminlerimizi yapalım
predictions = model.predict(test_features_matrix)

In [23]:
# Modelimizin isabet oranına bakalım
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(predictions, test_labels)
print('Accuracy of model: ', accuracy)

Accuracy of model:  0.9730769230769231
